### Imports

In [22]:
import pandas as pd
import numpy as np
from pandas import json_normalize
import matplotlib.pyplot as plt


#📌 Extracão

In [5]:
df = pd.read_json("TelecomX_Data.json")
df_transformed = df.copy()

#🔧 Transformação

In [7]:

dict_cols = [col for col in df_transformed.columns if isinstance(df_transformed[col].iloc[0], dict)]
for col in dict_cols:
    expanded = df_transformed[col].apply(pd.Series)
    expanded.columns = [f"{col}_{subcol}" for subcol in expanded.columns]
    df_transformed = pd.concat([df_transformed.drop(col, axis=1), expanded], axis=1)

""" 
Acabo realizando o processo duas vezes devido ao fato de 
haver dicionários dentro de dicionários no json.

"""

dict_cols = [col for col in df_transformed.columns if isinstance(df_transformed[col].iloc[0], dict)]
for col in dict_cols:
    expanded = df_transformed[col].apply(pd.Series)
    expanded.columns = [f"{col}_{subcol}" for subcol in expanded.columns]
    df_transformed = pd.concat([df_transformed.drop(col, axis=1), expanded], axis=1)



#📊 Carga e análise

In [8]:
duplicates = df_transformed['customerID'].duplicated(keep='first')
duplicates.value_counts()


customerID
False    7267
Name: count, dtype: int64

In [9]:
for i in df_transformed.columns:
   print(f"Número de NaN em {i} = {list(df_transformed[i].isna()).count(True)}")

Número de NaN em customerID = 0
Número de NaN em Churn = 0
Número de NaN em customer_gender = 0
Número de NaN em customer_SeniorCitizen = 0
Número de NaN em customer_Partner = 0
Número de NaN em customer_Dependents = 0
Número de NaN em customer_tenure = 0
Número de NaN em phone_PhoneService = 0
Número de NaN em phone_MultipleLines = 0
Número de NaN em internet_InternetService = 0
Número de NaN em internet_OnlineSecurity = 0
Número de NaN em internet_OnlineBackup = 0
Número de NaN em internet_DeviceProtection = 0
Número de NaN em internet_TechSupport = 0
Número de NaN em internet_StreamingTV = 0
Número de NaN em internet_StreamingMovies = 0
Número de NaN em account_Contract = 0
Número de NaN em account_PaperlessBilling = 0
Número de NaN em account_PaymentMethod = 0
Número de NaN em account_Charges_Monthly = 0
Número de NaN em account_Charges_Total = 0


In [10]:
df_transformed = df_transformed.convert_dtypes()
df_transformed.convert_dtypes().dtypes

customerID                   string[python]
Churn                        string[python]
customer_gender              string[python]
customer_SeniorCitizen                Int64
customer_Partner             string[python]
customer_Dependents          string[python]
customer_tenure                       Int64
phone_PhoneService           string[python]
phone_MultipleLines          string[python]
internet_InternetService     string[python]
internet_OnlineSecurity      string[python]
internet_OnlineBackup        string[python]
internet_DeviceProtection    string[python]
internet_TechSupport         string[python]
internet_StreamingTV         string[python]
internet_StreamingMovies     string[python]
account_Contract             string[python]
account_PaperlessBilling     string[python]
account_PaymentMethod        string[python]
account_Charges_Monthly             Float64
account_Charges_Total        string[python]
dtype: object

In [19]:
df_transformed['account_Charges_Daily'] = [ df_transformed['account_Charges_Monthly'][i]/30 for i in df_transformed.index]

In [38]:
data.std()

30.129571885666984

In [40]:
df_transformed.columns

Index(['customerID', 'Churn', 'customer_gender', 'customer_SeniorCitizen',
       'customer_Partner', 'customer_Dependents', 'customer_tenure',
       'phone_PhoneService', 'phone_MultipleLines', 'internet_InternetService',
       'internet_OnlineSecurity', 'internet_OnlineBackup',
       'internet_DeviceProtection', 'internet_TechSupport',
       'internet_StreamingTV', 'internet_StreamingMovies', 'account_Contract',
       'account_PaperlessBilling', 'account_PaymentMethod',
       'account_Charges_Monthly', 'account_Charges_Total', 'daily',
       'account_Charges_Daily'],
      dtype='object')

In [42]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Análise das Cobranças da Conta', fontsize=16)

dados = {
    'account_Charges_Monthly': {
        'title': 'Cobrança Mensal',
        'xlabel': 'Valor Mensal (R$)',
        'color': 'skyblue'
    },
    'account_Charges_Total': {
        'title': 'Cobrança Total',
        'xlabel': 'Valor Total (R$)',
        'color': 'salmon'
    },
    'account_Charges_Daily': {
        'title': 'Cobrança Diária',
        'xlabel': 'Valor Diário (R$)',
        'color': 'lightgreen'
    }
}

for ax, (col, config) in zip(axes, dados.items()):
    data = df_transformed[col].dropna()
    
    ax.hist(data, bins=15, edgecolor='black', color=config['color'], alpha=0.7)
    ax.grid(True, linestyle='--', alpha=0.6)
    
    ax.axvline(data.mean(), color='red', linestyle='-', linewidth=2, label=f'Média: R${data.mean():.2f}')
    ax.axvline(data.median(), color='blue', linestyle='--', linewidth=2, label=f'Mediana: R${data.median():.2f}')
    ax.axvline(data.std(), color='green', linestyle=':', linewidth=2, label=f'Desvio Padrão: R${data.std():.2f}')
    
    ax.set_title(config['title'])
    ax.set_xlabel(config['xlabel'])
    ax.set_ylabel('Frequência')
    ax.legend()

plt.tight_layout()
plt.show()

TypeError: Cannot perform reduction 'mean' with string dtype

#📄Relatorio Final